In [ ]:
import os


class Drivev4(Megadrive):
    def set(self):
        self.files = ['list.txt', 'path.txt', 'rclone.conf']

    def init(self):
        content = self.read_file('path.txt')
        if len(content) < 2:
            self.endmsg = 'Args corrupted'
            return False
        self.from_path = content[0]
        self.to_path = content[1]
        if not os.path.isfile('/usr/bin/rclone'):
            print('----- Installing rclone -----')
            !curl https://rclone.org/install.sh | sudo bash
        return True
    
    def check_phase(self):
        get_ipython().system_raw(f"rclone --config=/content/rclone.conf lsf 'FROM:{self.from_path}' --files-only -R | sort > src")
        get_ipython().system_raw(f"rclone --config=/content/rclone.conf lsf 'TO:{self.to_path}' --files-only -R | sort > dst")
        get_ipython().system_raw(f"comm -23 src dst > lst")
        get_ipython().system_raw("wc -l < src > line_src")
        get_ipython().system_raw("wc -l < lst > line_lst")
        with open('line_src', 'r') as f:
            total_file = int(f.read())
        with open('line_lst', 'r') as f:
            self.file_total = int(f.read())
        self.file_completed = total_file - self.file_total
        print(f'Checking done. {self.file_completed} files completed. {self.file_total} files on queue.')
        self.log('INFO', f'{self.file_completed} files completed. {self.file_total} files on queue.')
    
    def transfer_phase(self):
        get_ipython().system_raw(f"rclone --config=/content/rclone.conf copy 'FROM:{self.from_path}' 'TO:{self.to_path}' --files-from lst --no-check-certificate --drive-acknowledge-abuse -v")
